Depositos para enviar, orderm de preferencia:
musical_matriz
musical_filal
em seguida onde houver mais estoque


onde days_available = 0 criar observação onde informa que há muito tempo sem estoque

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [2]:
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

### Período a consultar

In [3]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 11, 1).date()
data_fim = datetime(2023, 12, 5).date()
data_fim = data_fim + timedelta(days=1)  # + 1 dia para pegar a data atual no DB
print(data_fim)

2023-12-06


### Historico de estoque

In [4]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM fulfillment_stock_hist WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim};'"
    # sql_query = f"SELECT * FROM fulfillment_stock_hist WHERE created_at BETWEEN '2023-12-04' AND '2023-12-05';"
    print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM fulfillment_stock_hist WHERE created_at BETWEEN '2023-11-01' AND '2023-12-06;'


C:\Users\couti\AppData\Local\Temp\ipykernel_16204\2278021421.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [77]:
df_stock.columns

Index(['ml_inventory_id', 'available_quantity', 'detail_status',
       'detail_quantity', 'references_id', 'references_variation_id', 'data',
       'has_stock', 'days_available'],
      dtype='object')

In [83]:
df_stock["data"].value_counts().sort_values(ascending=True)

data
2023-12-05    364
2023-12-01    635
2023-11-29    635
2023-11-28    635
2023-11-27    635
2023-11-25    635
2023-11-24    635
2023-11-23    635
2023-11-20    635
2023-11-19    635
2023-11-18    635
2023-11-17    635
2023-12-04    635
2023-11-01    635
2023-12-02    635
2023-11-14    635
2023-11-12    635
2023-11-11    635
2023-11-10    635
2023-11-09    635
2023-11-08    635
2023-11-07    635
2023-11-06    635
2023-11-05    635
2023-11-04    635
2023-11-03    635
2023-11-02    635
2023-11-15    635
2023-11-16    635
2023-11-13    635
2023-11-30    686
Name: count, dtype: int64

In [6]:
# ### Adicionando dados de estoque

# conn = psycopg2.connect(**db_config)
# cursor = conn.cursor()

# for index, row in df_stock.iterrows():
#     data_ontem = datetime.now() - timedelta(days=1)
#     insert_query = sql.SQL(
#         """
#         INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity,created_at)
#         VALUES (%s, %s, %s)
#     """
#     )
#     cursor.execute(insert_query, (row["ml_inventory_id"], row["available_quantity"],data_ontem))

# conn.commit()

# cursor.close()
# conn.close()

# print("Dados inseridos com sucesso!")

In [7]:
# datas consultadas, dias em que um produto pode ou não estar disponível
df_stock["created_at"].value_counts().index.to_list()

[Timestamp('2023-11-06 01:00:00'),
 Timestamp('2023-11-11 01:00:00'),
 Timestamp('2023-11-01 01:00:00'),
 Timestamp('2023-11-02 01:00:00'),
 Timestamp('2023-11-03 01:00:00'),
 Timestamp('2023-11-04 01:00:00'),
 Timestamp('2023-11-05 01:00:00'),
 Timestamp('2023-11-07 01:00:00'),
 Timestamp('2023-11-08 01:00:00'),
 Timestamp('2023-11-09 01:00:00'),
 Timestamp('2023-11-10 01:00:00'),
 Timestamp('2023-11-12 01:00:00'),
 Timestamp('2023-11-13 01:00:00'),
 Timestamp('2023-11-14 01:00:00'),
 Timestamp('2023-11-15 01:00:00'),
 Timestamp('2023-11-16 01:00:00'),
 Timestamp('2023-11-17 01:00:00'),
 Timestamp('2023-11-18 01:00:00'),
 Timestamp('2023-11-19 01:00:00'),
 Timestamp('2023-11-20 01:00:00'),
 Timestamp('2023-11-28 17:24:31.935643'),
 Timestamp('2023-12-02 00:01:03.175701'),
 Timestamp('2023-12-01 00:01:02.530600'),
 Timestamp('2023-11-30 00:01:02.863494'),
 Timestamp('2023-11-29 00:01:02.668739'),
 Timestamp('2023-12-04 00:01:03.041159'),
 Timestamp('2023-11-27 00:31:03.121363'),
 Times

In [8]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by="created_at", ascending=False)
df_stock["data"] = df_stock["created_at"].dt.date
df_stock = df_stock.drop(["created_at"], axis=1)

df_stock

,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data
0,AVOX05934,7,None,NaN,MLB2003190501,nan,2023-12-05
251,ZUIU17173,1,None,NaN,MLB2025558078,173669630956,2023-12-05
249,XYRN72687,2,None,NaN,MLB2024002438,17366405652,2023-12-05
248,LESW73286,5,None,NaN,MLB2024002438,173664056522,2023-12-05
247,DCGQ72320,2,None,NaN,MLB2024002438,173664056521,2023-12-05
...,...,...,...,...,...,...,...
26185,OWZF76125,0,None,NaN,None,None,2023-11-01
26184,SDNY53354,2,None,NaN,None,None,2023-11-01
26183,KMCL67683,5,None,NaN,None,None,2023-11-01
26182,KBLG86592,0,None,NaN,None,None,2023-11-01


In [9]:
# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity

condicao = df_stock["detail_status"] == "transfer"

df_stock.loc[condicao, "available_quantity"] += df_stock.loc[
    condicao, "detail_quantity"
]
df_stock

,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data
0,AVOX05934,7,None,NaN,MLB2003190501,nan,2023-12-05
251,ZUIU17173,1,None,NaN,MLB2025558078,173669630956,2023-12-05
249,XYRN72687,2,None,NaN,MLB2024002438,17366405652,2023-12-05
248,LESW73286,5,None,NaN,MLB2024002438,173664056522,2023-12-05
247,DCGQ72320,2,None,NaN,MLB2024002438,173664056521,2023-12-05
...,...,...,...,...,...,...,...
26185,OWZF76125,0,None,NaN,None,None,2023-11-01
26184,SDNY53354,2,None,NaN,None,None,2023-11-01
26183,KMCL67683,5,None,NaN,None,None,2023-11-01
26182,KBLG86592,0,None,NaN,None,None,2023-11-01


In [10]:
condf(df_stock, "detail_status", "transfer")

,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data
246,UMNA18288,2,transfer,1.0,MLB2024265635,173664488135,2023-12-05
244,POPO18550,7,transfer,2.0,MLB2024265635,173664488133,2023-12-05
231,VUJI10625,21,transfer,2.0,MLB2006357183,nan,2023-12-05
250,JPOV78690,3,transfer,1.0,MLB2024002438,173664056523,2023-12-05
199,ELQK04205,115,transfer,23.0,MLB1992889189,173551307783,2023-12-05
191,YDQA03755,19,transfer,5.0,MLB2001055363,nan,2023-12-05
186,OLCT05266,33,transfer,14.0,MLB2001110525,nan,2023-12-05
204,BPGQ06190,79,transfer,15.0,MLB2003283466,nan,2023-12-05
217,HSDO10298,9,transfer,1.0,MLB2004848109,nan,2023-12-05
213,NHQO05910,23,transfer,3.0,MLB2003138943,nan,2023-12-05


In [11]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by="data", ascending=False).reset_index(drop=True)
df_stock

,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock
0,AVOX05934,7,None,NaN,MLB2003190501,nan,2023-12-05,True
1,ZZGN21450,5,None,NaN,MLB1403999905,nan,2023-12-05,True
2,JLAX10536,3,None,NaN,MLB2004809736,nan,2023-12-05,True
3,SJLM76488,5,None,NaN,MLB2001160804,nan,2023-12-05,True
4,JFGN34621,0,lost,1.0,MLB924922735,nan,2023-12-05,False
...,...,...,...,...,...,...,...,...
32741,JJSZ06277,48,None,NaN,None,None,2023-11-01,True
32742,DSGP06979,7,None,NaN,None,None,2023-11-01,True
32743,DSGP06967,6,None,NaN,None,None,2023-11-01,True
32744,BLHH53768,4,None,NaN,None,None,2023-11-01,True


In [12]:
condf(df_stock, "ml_inventory_id", "DSGP06967")

,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock
168,DSGP06967,5,None,NaN,MLB1992567302,173585284762,2023-12-05,True
578,DSGP06967,6,None,NaN,None,None,2023-12-04,True
1215,DSGP06967,9,None,NaN,None,None,2023-12-02,True
2186,DSGP06967,11,None,NaN,None,None,2023-12-01,True
2899,DSGP06967,7,None,NaN,None,None,2023-11-30,True
3533,DSGP06967,11,None,NaN,None,None,2023-11-30,True
3974,DSGP06967,9,None,NaN,None,None,2023-11-29,True
4489,DSGP06967,9,None,NaN,None,None,2023-11-28,True
5439,DSGP06967,6,None,NaN,None,None,2023-11-27,True
6061,DSGP06967,6,None,NaN,None,None,2023-11-25,True


In [13]:
df_stock = df_stock.drop_duplicates()

In [14]:
condf(df_stock, "ml_inventory_id", "DSGP06967")
df_stock["ml_inventory_id"].value_counts()
len(df_stock["ml_inventory_id"].unique())
df_stock["ml_inventory_id"].unique()

array(['AVOX05934', 'ZZGN21450', 'JLAX10536', 'SJLM76488', 'JFGN34621',
       'TDFV00990', 'QYGO52123', 'ERIM51807', 'DGBX50777', 'TDFV51283',
       'PKNN52962', 'IXGP53982', 'OYUK51882', 'CCQW73335', 'DXJZ75064',
       'JDMV76187', 'RJUL81287', 'CIKE94068', 'MENA20785', 'HXJD21466',
       'CQAQ20361', 'GLMK21133', 'ITMG31457', 'XGVG11805', 'TXWS20511',
       'DCPP20168', 'FVTQ20181', 'HTEZ21817', 'XYHC31118', 'SCKP53673',
       'SDNY53354', 'BOWK32167', 'ZZRV30730', 'KRJP31426', 'SCVF31712',
       'MLVG32506', 'IRUD34507', 'TPDH38567', 'TLCG38735', 'PUOF38682',
       'ZXFT21237', 'JGRW38703', 'AIRQ38676', 'GPGU38258', 'ABCB20467',
       'XDTG30724', 'BPGQ10723', 'EDGP47752', 'STLG62905', 'VGBX48763',
       'SKXW47953', 'KBOK48388', 'PROW53057', 'WOXA67988', 'YLXH65172',
       'CNHG47042', 'PVKY65418', 'BOWK48319', 'XYHC65769', 'XGSW66673',
       'KNPR48414', 'SKXW65690', 'HXJD67665', 'WIZE67325', 'CNHG66088',
       'CJBY66878', 'PWQU66026', 'KMCL67683', 'NWXH48678', 'QOGG

#### Dias em que produto esteve disponível

In [15]:
## Contando dias em que produto esteve disponível
days_available = df_stock.groupby(["ml_inventory_id"])["has_stock"].sum().reset_index()
days_available = days_available.rename(columns={"has_stock": "days_available"})
days_available

,ml_inventory_id,days_available
0,ABCB20467,31
1,ABCB48480,0
2,ACBY18022,0
3,ADJX46855,0
4,ADRN17806,0
...,...,...
641,ZXJO91694,31
642,ZZGN21450,31
643,ZZQL21216,8
644,ZZRV30730,31


In [16]:
condf(df_stock, "ml_inventory_id", "ABCB20467")

,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock
44,ABCB20467,9,None,NaN,MLB1403024540,48941021297,2023-12-05,True
646,ABCB20467,9,None,NaN,None,None,2023-12-04,True
1282,ABCB20467,10,None,NaN,None,None,2023-12-02,True
1920,ABCB20467,10,None,NaN,None,None,2023-12-01,True
2620,ABCB20467,10,None,NaN,None,None,2023-11-30,True
3817,ABCB20467,10,None,NaN,None,None,2023-11-29,True
4526,ABCB20467,10,None,NaN,None,None,2023-11-28,True
5087,ABCB20467,10,None,NaN,None,None,2023-11-27,True
5724,ABCB20467,10,None,NaN,None,None,2023-11-25,True
6357,ABCB20467,10,None,NaN,None,None,2023-11-24,True


In [17]:
# ## Contando dias em que produto esteve disponível
# days_available = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available = days_available.rename(columns={"has_stock": "days_available"})
# days_available

In [18]:
condf(days_available, "ml_inventory_id", "DSGP06967")
# condf(df_stock, "ml_inventory_id", "DSGP06967")

,ml_inventory_id,days_available
81,DSGP06967,32


In [19]:
# Unindo DFs
df_stock = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# df_stock.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# df_stock = df_stock.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# Verificando o formato após a correção
print(df_stock.shape)
df_stock

(19465, 9)


,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
0,AVOX05934,7,None,NaN,MLB2003190501,nan,2023-12-05,True,31
1,AVOX05934,7,None,NaN,None,None,2023-12-04,True,31
2,AVOX05934,7,None,NaN,None,None,2023-12-02,True,31
3,AVOX05934,7,None,NaN,None,None,2023-12-01,True,31
4,AVOX05934,7,None,NaN,None,None,2023-11-30,True,31
...,...,...,...,...,...,...,...,...,...
19460,EJRL85435,0,None,NaN,None,None,2023-11-05,False,0
19461,EJRL85435,0,None,NaN,None,None,2023-11-04,False,0
19462,EJRL85435,0,None,NaN,None,None,2023-11-03,False,0
19463,EJRL85435,0,None,NaN,None,None,2023-11-02,False,0


In [20]:
df_stock

,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
0,AVOX05934,7,None,NaN,MLB2003190501,nan,2023-12-05,True,31
1,AVOX05934,7,None,NaN,None,None,2023-12-04,True,31
2,AVOX05934,7,None,NaN,None,None,2023-12-02,True,31
3,AVOX05934,7,None,NaN,None,None,2023-12-01,True,31
4,AVOX05934,7,None,NaN,None,None,2023-11-30,True,31
...,...,...,...,...,...,...,...,...,...
19460,EJRL85435,0,None,NaN,None,None,2023-11-05,False,0
19461,EJRL85435,0,None,NaN,None,None,2023-11-04,False,0
19462,EJRL85435,0,None,NaN,None,None,2023-11-03,False,0
19463,EJRL85435,0,None,NaN,None,None,2023-11-02,False,0


In [21]:
# data de hoje
# data_de_hoje = datetime.now().date() - timedelta(days=1)
# print(data_de_hoje)
data_de_hoje = datetime.now().date()
print(data_de_hoje)
df_stock["data"] = pd.to_datetime(df_stock["data"])
# data_de_hoje = data_de_hoje + timedelta(days=1)  # + 1 dia para pegar a data atual no DB

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock[df_stock["data"].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(
    columns={"available_quantity": "available_quantity_today"}
)
# df_stock_today = df_stock.drop(['has_stock'], axis=1)
df_stock_today

2023-12-05


,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
0,AVOX05934,7,None,NaN,MLB2003190501,nan,2023-12-05,True,31
31,ZZGN21450,5,None,NaN,MLB1403999905,nan,2023-12-05,True,31
62,JLAX10536,3,None,NaN,MLB2004809736,nan,2023-12-05,True,31
93,SJLM76488,5,None,NaN,MLB2001160804,nan,2023-12-05,True,1
94,JFGN34621,0,lost,1.0,MLB924922735,nan,2023-12-05,False,0
...,...,...,...,...,...,...,...,...,...
10850,IYNL59473,8,None,NaN,MLB3339660079,178533059695,2023-12-05,True,31
10881,FXMN50419,0,None,NaN,MLB950296037,nan,2023-12-05,False,0
10912,PUMV59027,2,None,NaN,MLB3696018282,178531159087,2023-12-05,True,31
10943,IDSV51539,64,None,NaN,MLB3339660079,178533059699,2023-12-05,True,31


In [22]:
df_stock_today

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
0,AVOX05934,7,None,NaN,MLB2003190501,nan,2023-12-05,True,31
31,ZZGN21450,5,None,NaN,MLB1403999905,nan,2023-12-05,True,31
62,JLAX10536,3,None,NaN,MLB2004809736,nan,2023-12-05,True,31
93,SJLM76488,5,None,NaN,MLB2001160804,nan,2023-12-05,True,1
94,JFGN34621,0,lost,1.0,MLB924922735,nan,2023-12-05,False,0
...,...,...,...,...,...,...,...,...,...
10850,IYNL59473,8,None,NaN,MLB3339660079,178533059695,2023-12-05,True,31
10881,FXMN50419,0,None,NaN,MLB950296037,nan,2023-12-05,False,0
10912,PUMV59027,2,None,NaN,MLB3696018282,178531159087,2023-12-05,True,31
10943,IDSV51539,64,None,NaN,MLB3339660079,178533059699,2023-12-05,True,31


In [23]:
x = df_stock_today[df_stock_today["detail_status"] == "transfer"]
x

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
563,ITMG31457,1,transfer,1.0,MLB1417877602,nan,2023-12-05,True,4
750,XYHC31118,2,transfer,1.0,MLB1416821703,nan,2023-12-05,True,31
781,SCKP53673,12,transfer,1.0,MLB1418498416,nan,2023-12-05,True,31
1527,WOXA67988,15,transfer,3.0,MLB1461223727,53864903711,2023-12-05,True,26
1840,WIZE67325,2,transfer,1.0,MLB1461563722,nan,2023-12-05,True,23
2184,JNYZ66723,14,transfer,1.0,MLB1465231671,52179893239,2023-12-05,True,31
2246,LDBL64899,21,transfer,8.0,MLB1465231671,52180354992,2023-12-05,True,31
2775,WTQW54410,1,transfer,1.0,MLB1629453392,nan,2023-12-05,True,29
3616,YQWM23606,10,transfer,1.0,MLB1737406124,nan,2023-12-05,True,31
3680,UIUW89800,16,transfer,12.0,MLB1662970458,64498487033,2023-12-05,True,32


### Buscando hitorico de orders no BD

In [24]:
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM ml_orders_hist WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# filtros
df_orders = df_orders[df_orders["fulfilled"] == True]
df_orders = df_orders[df_orders["order_status"] == "paid"]
df_orders = df_orders[df_orders["payment_status"] == "approved"]
df_orders = df_orders.drop(
    ["pack_id", "date_approved", "fulfilled", "order_status", "payment_status"], axis=1
)
df_orders.rename(columns={"quantity": "sold_quantity"}, inplace=True)

SELECT * FROM ml_orders_hist WHERE date_closed BETWEEN '2023-11-01' AND '2023-12-06'


C:\Users\couti\AppData\Local\Temp\ipykernel_16204\1771993387.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql(sql_query, conn)


In [25]:
df_orders.sample()

,ml_code,category_id,variation_id,seller_sku,sold_quantity,title,shipping_id,date_closed
6973,MLB2770292345,MLB278096,175608161444,FULLVANDOREN2SR212,1,"2 Palheta Sax Alto Vandoren Tradicional 1 1,5 ...",42786806958,2023-11-10 17:33:10


In [26]:
# Ordenando orders por data
df_orders = df_orders.sort_values(by="date_closed", ascending=False)
df_orders["data"] = df_orders["date_closed"].dt.date
df_orders = df_orders.drop(["date_closed"], axis=1)

print(df_orders.shape)
df_orders.head(3)

(5698, 8)


,ml_code,category_id,variation_id,seller_sku,sold_quantity,title,shipping_id,data
10696,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42867562004,2023-12-04
5188,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42867562004,2023-12-04
5176,MLB4086313280,MLB45629,nan,EN-1322-PS,1,Pele Bumbo Encore 22 Pinstripe Transparente En...,42867181643,2023-12-04


In [27]:
df_orders = df_orders.drop_duplicates()
df_orders.shape

(2849, 8)

In [28]:
condf(df_orders, "ml_code", "MLB1992541482")

,ml_code,category_id,variation_id,seller_sku,sold_quantity,title,shipping_id,data
8662,MLB1992541482,MLB278076,173552533267,FULLSG6687,3,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42824583640,2023-11-21
3038,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42822412452,2023-11-21
2630,MLB1992541482,MLB278076,173552533264,FULLSG6684,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42812680605,2023-11-18
2366,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42806017696,2023-11-16
7725,MLB1992541482,MLB278076,173552533264,FULLSG6684,2,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42802756417,2023-11-15
7595,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42799541207,2023-11-14
2085,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42799607306,2023-11-14
2070,MLB1992541482,MLB278076,173552533266,FULLSG6686,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42799272805,2023-11-14
7315,MLB1992541482,MLB278076,173552533267,FULLSG6687,3,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42793563096,2023-11-13
6643,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,42779504189,2023-11-08


#### Total de vendas por ml_code e variation_id

In [29]:
df_orders.shape

(2849, 8)

In [30]:
# Verificar se a coluna variation_id existe
condicao_variation = df_orders["variation_id"].notna()

# Agrupar por variation_id ou ml_code, dependendo da condição
# grupo_coluna = 'variation_id' if condicao_variation.any() else 'ml_code'

# Somar sold_quantity com base na regra
resultado = (
    df_orders.groupby(["ml_code", "variation_id"])["sold_quantity"].sum().reset_index()
)

# Exibir o resultado
# resultado['variation_id	'].value_counts()

In [31]:
condf(df_orders, "ml_code", "MLB1263116172")

,ml_code,category_id,variation_id,seller_sku,sold_quantity,title,shipping_id,data
7269,MLB1263116172,MLB29341,nan,DS510BC,1,Suporte Profissional Para Trompete Hercules Ds...,42792277963,2023-11-12


In [32]:
resultado

,ml_code,variation_id,sold_quantity
0,MLB1263116172,nan,1
1,MLB1266619992,39941229352,1
2,MLB1267686157,39909127116,6
3,MLB1344733856,nan,3
4,MLB1344736999,nan,3
...,...,...,...
529,MLB931902349,24312282395,1
530,MLB931902349,24312282400,1
531,MLB931902349,24321463460,1
532,MLB950214971,nan,1


In [33]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(
    # df_orders, resultado, on=["ml_code", "variation_id"], how="inner"
    df_orders,
    resultado,
    on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales = df_total_sales.rename(
    columns={"sold_quantity_y": "total_sold_quantity"}
)
df_total_sales.shape

(2849, 9)

In [34]:
df_total_sales.head(3)

,ml_code,category_id,variation_id,seller_sku,sold_quantity_x,title,shipping_id,data,total_sold_quantity
0,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42867562004,2023-12-04,40
1,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42864582492,2023-12-03,40
2,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,2,Pandeiro Meia Lua Pml001 Preto Liverpool,42822712399,2023-11-21,40


In [35]:
condf(df_total_sales, "ml_code", "MLB4127982748")

,ml_code,category_id,variation_id,seller_sku,sold_quantity_x,title,shipping_id,data,total_sold_quantity
0,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42867562004,2023-12-04,40
1,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42864582492,2023-12-03,40
2,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,2,Pandeiro Meia Lua Pml001 Preto Liverpool,42822712399,2023-11-21,40
3,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42822537227,2023-11-21,40
4,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,3,Pandeiro Meia Lua Pml001 Preto Liverpool,42822015892,2023-11-21,40
5,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,4,Pandeiro Meia Lua Pml001 Preto Liverpool,42821845689,2023-11-21,40
6,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42819852739,2023-11-20,40
7,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42814925734,2023-11-19,40
8,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42814385093,2023-11-19,40
9,MLB4127982748,MLB29319,nan,FULLLIVERPOOLPMLTENC,1,Pandeiro Meia Lua Pml001 Preto Liverpool,42814414064,2023-11-19,40


In [36]:
df_total_sales = df_total_sales.drop(["sold_quantity_x", "shipping_id", "data"], axis=1)
df_total_sales = df_total_sales.drop_duplicates()

In [37]:
condf(df_total_sales, "ml_code", "MLB1992541482")

,ml_code,category_id,variation_id,seller_sku,title,total_sold_quantity
929,MLB1992541482,MLB278076,173552533267,FULLSG6687,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,15
1823,MLB1992541482,MLB278076,173552533264,FULLSG6684,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,4
2240,MLB1992541482,MLB278076,173552533266,FULLSG6686,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 8...,1


Neste ponto temos o total de itens vendidos de um anúncio por período e a quantidade de dias em que um produto esteve disponível.
precisamos juntar esses dados para calcular, para isso trarei as informações de produtos

In [38]:
print(df_total_sales.shape)
df_total_sales.sample()

(534, 6)


,ml_code,category_id,variation_id,seller_sku,title,total_sold_quantity
2074,MLB3943436920,MLB29319,nan,C48291FIZZO,Pandeiro Profissional 11 Pol Pele Prismatica C...,4


In [39]:
print(df_stock_today.shape)
df_stock_today.sample()

(364, 9)


,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
9199,UWGL04723,0,None,NaN,MLB2660440797,17457982183,2023-12-05,False,27


#### Buscando Produtos

In [40]:
# Buscando dados de produtos na tabela tiny_fulfillment
try:
    conn = psycopg2.connect(**db_config)

    sql_query = "SELECT * FROM items"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar tiny_fulfillment: {e}")

except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

# df_codes["ml_code"] = df_codes["ml_code"].apply(lambda x: "MLB" + str(x))
df_codes.rename(columns={"inventory_id": "ml_inventory_id"}, inplace=True)
df_codes = df_codes.drop(["created_at", "updated_at"], axis=1)

C:\Users\couti\AppData\Local\Temp\ipykernel_16204\1903207410.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


In [41]:
df_not_catalogo = df_codes[df_codes["catalog_listing"] == False]
df_catalogo = df_codes[df_codes["catalog_listing"] == True]
df_not_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing
63,MLB1461638936,HXJD67665,Preto,51963606356,paused,False


In [42]:
df_catalogo.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing
408,MLB3387613287,HSDO10298,NaN,NaN,active,True


In [43]:
df_vendas_cat = pd.merge(
    df_catalogo, df_orders, left_on=["ml_code"], right_on=["ml_code"], how="inner"
)
df_vendas_not_cat = pd.merge(
    df_not_catalogo,
    df_orders,
    left_on=["ml_code", "variation_id"],
    right_on=["ml_code", "variation_id"],
    how="inner",
)
df_vendas_cat.sample(3)

,ml_code,ml_inventory_id,value_name,variation_id_x,status,catalog_listing,category_id,variation_id_y,seller_sku,sold_quantity,title,shipping_id,data
327,MLB3829652726,ZOQD52873,NaN,NaN,active,True,MLB72745,nan,FULLP10NINJA15FT,1,"Cabo Santo Angelo Ninja 4,57m P10 Reto Guitarr...",42797834414,2023-11-14
113,MLB3778537856,BPGQ06190,NaN,NaN,active,True,MLB72752,nan,FULLBASSOVTFL01,1,Correia Guitarra Violão Baixo Basso Alça Vt Fl...,42812411156,2023-11-18
534,MLB3900020030,RXXA58942,NaN,NaN,active,True,MLB29319,nan,FULLLIVERPOOLPML001A,1,Pandeiro Profissional Meia Lua Diversas Cores ...,42800274439,2023-11-15


In [44]:
df_vendas_not_cat.sample(3)

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,category_id,seller_sku,sold_quantity,title,shipping_id,data
259,MLB1992889189,ELQK04205,3 PARES 5B,173551307783,active,False,MLB429603,FULLLIVERPOOL3NI5BM,2,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,42765519105,2023-11-04
228,MLB1992889189,ELQK04205,3 PARES 5B,173551307783,active,False,MLB429603,FULLLIVERPOOL3NI5BM,1,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,42788033358,2023-11-11
296,MLB1992889189,FSNB97805,3 PARES 5A,173551307782,active,False,MLB429603,FULLLIVERPOOL3NI5AM,2,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,42777087766,2023-11-08


In [45]:
resultado_cat = (
    df_vendas_cat.groupby("ml_inventory_id")["sold_quantity"].sum().reset_index()
)

resultado_cat

,ml_inventory_id,sold_quantity
0,AGDL67547,2
1,AJZG23550,3
2,AWAZ99732,1
3,AZVM53003,1
4,BPGQ06190,40
...,...,...
89,ZJTW10331,2
90,ZOQD52873,288
91,ZQIV51979,11
92,ZZGN21450,1


In [46]:
resultado_not_cat = (
    df_vendas_not_cat.groupby("ml_inventory_id")["sold_quantity"].sum().reset_index()
)
resultado_not_cat

,ml_inventory_id,sold_quantity
0,ARDK78657,10
1,BOWK48319,3
2,BSHY44610,9
3,BZGW44149,1
4,BZGW51616,9
...,...,...
86,YXIW04349,29
87,ZRTG53907,1
88,ZUIU17173,1
89,ZUKG50028,28


In [85]:
condf(resultado_not_cat, "ml_inventory_id", "AGDL67547")
# condf(resultado_cat, 'ml_inventory_id', 'AGDL67547')

,ml_inventory_id,sold_quantity


In [47]:
print(resultado_cat.shape)
print(resultado_not_cat.shape)

(94, 2)
(91, 2)


In [48]:
resultado = pd.merge(
    resultado_cat, resultado_not_cat, on=["ml_inventory_id"], how="outer"
)
resultado = resultado.fillna(0)
# pd.set_option('display.max_rows', None)
pd.reset_option("display.max_rows")

resultado["sold_quantity_sum"] = resultado["sold_quantity_x"].fillna(0) + resultado[
    "sold_quantity_y"
].fillna(0)
df_total_sales = resultado.copy()

In [49]:
df_total_sales

,ml_inventory_id,sold_quantity_x,sold_quantity_y,sold_quantity_sum
0,AGDL67547,2.0,0.0,2.0
1,AJZG23550,3.0,0.0,3.0
2,AWAZ99732,1.0,0.0,1.0
3,AZVM53003,1.0,0.0,1.0
4,BPGQ06190,40.0,0.0,40.0
...,...,...,...,...
178,YXIW04349,0.0,29.0,29.0
179,ZRTG53907,0.0,1.0,1.0
180,ZUIU17173,0.0,1.0,1.0
181,ZUKG50028,0.0,28.0,28.0


In [50]:
df_codes.sample()

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing
489,MLB3474485135,GMII66042,NaN,NaN,paused,True


### Produtos + Dias disponíveis

In [51]:
print(df_codes.shape)
print(df_stock_today.shape)

print(df_codes.columns)
print(df_stock_today.columns)

(526, 6)
(364, 9)
Index(['ml_code', 'ml_inventory_id', 'value_name', 'variation_id', 'status',
       'catalog_listing'],
      dtype='object')
Index(['ml_inventory_id', 'available_quantity_today', 'detail_status',
       'detail_quantity', 'references_id', 'references_variation_id', 'data',
       'has_stock', 'days_available'],
      dtype='object')


In [53]:
# condf(prod_day, 'ml_code', 'MLB2003190501')

In [54]:
prod_day = pd.merge(df_codes, df_stock_today, on="ml_inventory_id", how="inner")
# prod_day = pd.merge(df_codes, df_stock_today, on="ml_inventory_id", how="outer")
prod_day = prod_day[prod_day["catalog_listing"] == False]
prod_day.shape
prod_day

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
1,MLB2003190501,AVOX05934,NaN,NaN,active,False,7,None,NaN,MLB2003190501,nan,2023-12-05,True,31
3,MLB2000971378,WTHI04163,NaN,NaN,active,False,15,transfer,1.0,MLB2000971378,nan,2023-12-05,True,32
6,MLB2004809736,JLAX10536,NaN,NaN,active,False,3,None,NaN,MLB2004809736,nan,2023-12-05,True,31
7,MLB924922735,JFGN34621,NaN,NaN,paused,False,0,lost,1.0,MLB924922735,nan,2023-12-05,False,0
9,MLB949771924,TDFV00990,NaN,NaN,paused,False,0,None,NaN,MLB949771924,nan,2023-12-05,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,MLB3339660079,IYNL59473,3,178533059695,active,False,8,None,NaN,MLB3339660079,178533059695,2023-12-05,True,31
519,MLB3339660079,ARDK78657,2,178533059697,active,False,15,None,NaN,MLB3339660079,178533059697,2023-12-05,True,31
520,MLB3339660079,IDSV51539,"2,5",178533059699,active,False,64,None,NaN,MLB3339660079,178533059699,2023-12-05,True,31
521,MLB3339660079,BZGW51616,"1,5",178533059701,active,False,25,None,NaN,MLB3339660079,178533059701,2023-12-05,True,31


### Prod_Day + Total_sales

In [55]:
print(df_total_sales.shape)
df_total_sales = df_total_sales.drop(["sold_quantity_x", "sold_quantity_y"], axis=1)
df_total_sales.sample()

(183, 4)


,ml_inventory_id,sold_quantity_sum
112,ELQK04205,84.0


In [59]:
# df_sales = pd.merge(
#     df_total_sales,
#     prod_day,
#     left_on=["ml_code", "variation_id"],
#     right_on=["ml_code", "variation_id"],
#     how="inner",
# )

df_sales = pd.merge(
    df_total_sales,
    prod_day,
    left_on=["ml_inventory_id"],
    right_on=["ml_inventory_id"],
    how="inner",
)

cols = [
    "ml_code",
    "ml_inventory_id",
    "sold_quantity_sum",
    "value_name",
    "variation_id",
    "status",
    "catalog_listing",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "references_id",
    "references_variation_id",
    "data",
    "has_stock",
    "days_available",
]

# df_sales = df_sales[cols]
# df_sales = df_sales.rename(columns={'days_available_x':'days_available'})
print(df_total_sales.shape)
print(prod_day.shape)
# print(x.shape)
print(df_sales.shape)

(183, 2)
(361, 14)
(183, 15)


In [60]:
df_sales.columns

Index(['ml_inventory_id', 'sold_quantity_sum', 'ml_code', 'value_name',
       'variation_id', 'status', 'catalog_listing', 'available_quantity_today',
       'detail_status', 'detail_quantity', 'references_id',
       'references_variation_id', 'data', 'has_stock', 'days_available'],
      dtype='object')

In [62]:
df_not_sales = pd.merge(
    df_total_sales,
    prod_day,
    left_on=["ml_inventory_id"],
    right_on=["ml_inventory_id"],
    how="inner",
)
# df_not_sales = df_not_sales[df_not_sales['days_available']==0]

cols = [
    "ml_code",
    "ml_inventory_id",
    "sold_quantity_sum",
    "value_name",
    "variation_id",
    "status",
    "catalog_listing",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "references_id",
    "references_variation_id",
    "data",
    "has_stock",
    "days_available",
]

df_not_sales = df_not_sales[cols]
# df_not_sales = df_not_sales.rename(columns={'days_available_x':'days_available'})
print(df_total_sales.shape)
print(prod_day.shape)
# print(x.shape)
print(df_not_sales.shape)

(183, 2)
(361, 14)
(183, 15)


In [63]:
# df_not_sales
df_sales.shape

(183, 15)

In [64]:
condf(df_sales, "ml_code", "MLB1992541482")

,ml_inventory_id,sold_quantity_sum,ml_code,value_name,variation_id,status,catalog_listing,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
100,CLMF99895,15.0,MLB1992541482,0.12,173552533267,active,False,11,lost,2.0,MLB1992541482,173552533267,2023-12-05,True,31
135,LUHT04035,4.0,MLB1992541482,0.09,173552533264,active,False,19,None,NaN,MLB1992541482,173552533264,2023-12-05,True,31
169,XGNL04251,1.0,MLB1992541482,0.11,173552533266,active,False,5,None,NaN,MLB1992541482,173552533266,2023-12-05,True,31


In [65]:
# x = pd.merge(x, df_sales, how='outer', indicator=True)
# x = x[x['_merge'] == 'right_only']

# x

### Calculando métricas

In [66]:
x = df_sales[df_sales["ml_inventory_id"] == "AGDL67547"]
x

,ml_inventory_id,sold_quantity_sum,ml_code,value_name,variation_id,status,catalog_listing,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
0,AGDL67547,2.0,MLB1465231671,Rosa,52181037451,active,False,15,None,NaN,MLB1465231671,52181037451,2023-12-05,True,7


In [67]:
df_sales.sample(10)

,ml_inventory_id,sold_quantity_sum,ml_code,value_name,variation_id,status,catalog_listing,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
83,YJPM55154,12.0,MLB2634456370,NaN,NaN,active,False,16,None,NaN,MLB2634456370,nan,2023-12-05,True,31
145,PQCQ36986,5.0,MLB2075516288,3 PARES 5A - 3 PARES 7A,173887190665,active,False,15,None,NaN,MLB2075516288,173887190665,2023-12-05,True,31
7,CJBY66878,3.0,MLB1461518341,NaN,NaN,active,False,6,None,NaN,MLB1461518341,nan,2023-12-05,True,32
23,ITQQ97036,21.0,MLB2001015502,NaN,NaN,paused,False,0,None,NaN,MLB2001015502,nan,2023-12-05,False,30
103,CTZK75126,1.0,MLB3696018282,1,178531159091,active,False,1,None,NaN,MLB3696018282,178531159091,2023-12-05,True,31
126,ICRX50305,13.0,MLB2770292345,"2,5",175608213909,active,False,6,None,NaN,MLB2770292345,175608213909,2023-12-05,True,31
175,YJSN05311,1.0,MLB2660440797,011-052,174579427304,active,False,2,None,NaN,MLB2660440797,174579427304,2023-12-05,True,31
60,RLJX42857,40.0,MLB3696114466,Preto,178530883033,active,False,41,transfer,8.0,MLB3696114466,178530883033,2023-12-05,True,32
44,MHIJ54971,9.0,MLB2636549148,NaN,NaN,active,False,14,transfer,1.0,MLB2636549148,nan,2023-12-05,True,1
151,SOQF03130,10.0,MLB2606294854,"2,5",174383874147,active,False,21,transfer,1.0,MLB2606294854,174383874147,2023-12-05,True,30


In [68]:
# media de produtos disponiveis no período
df_sales["media_prod_days_available"] = (
    df_sales["sold_quantity_sum"] / df_sales["days_available"]
)
df_sales["media_prod_days_available"] = df_sales["media_prod_days_available"].fillna(0)

days = 30

# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df_sales["period_send_fulfillment"] = np.ceil(
    (df_sales["sold_quantity_sum"] / df_sales["days_available"]) * days
    - df_sales["available_quantity_today"]
)
df_sales["period_send_fulfillment"] = df_sales["period_send_fulfillment"].fillna(0)

# qtd de produtos a enviar hoje, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df_sales["today_send_fulfillment"] = np.ceil(
    (df_sales["sold_quantity_sum"] / df_sales["days_available"])
    - df_sales["available_quantity_today"]
)
df_sales["today_send_fulfillment"] = df_sales["today_send_fulfillment"].fillna(0)

In [69]:
pd.set_option("display.max_columns", None)

In [70]:
import pandas as pd

# DataFrame da esquerda
df_esquerda = pd.DataFrame({"ID": [1, 2, 3, 4], "Valor_esquerda": ["A", "B", "C", "D"]})

# DataFrame da direita
df_direita = pd.DataFrame({"ID": [3, 4, 5, 6], "Valor_direita": ["X", "Y", "Z", "W"]})

# Realizando a junção anti-esquerda
resultado = pd.merge(
    df_esquerda, df_direita, on="ID", how="left", indicator=True
).query('_merge == "left_only"')

# Removendo a coluna de indicador (_merge)
resultado = resultado.drop(columns=["_merge"])

print(resultado)

   ID Valor_esquerda Valor_direita
0   1              A           NaN
1   2              B           NaN


In [71]:
cols = [
    "ml_code",
    "ml_inventory_id",
    "value_name",
    "status",
    "available_quantity_today",
    "sold_quantity_sum",
    "days_available",
    "media_prod_days_available",
    "period_send_fulfillment",
    "today_send_fulfillment",
]
df_sales = df_sales[cols]
df_sales

,ml_code,ml_inventory_id,value_name,status,available_quantity_today,sold_quantity_sum,days_available,media_prod_days_available,period_send_fulfillment,today_send_fulfillment
0,MLB1465231671,AGDL67547,Rosa,active,15,2.0,7,0.285714,-6.0,-14.0
1,MLB2004843162,AJZG23550,NaN,active,3,3.0,31,0.096774,-0.0,-2.0
2,MLB2001191174,AWAZ99732,NaN,active,15,1.0,28,0.035714,-13.0,-14.0
3,MLB1629586687,AZVM53003,NaN,active,9,1.0,31,0.032258,-8.0,-8.0
4,MLB2003283466,BPGQ06190,NaN,active,79,40.0,32,1.250000,-41.0,-77.0
...,...,...,...,...,...,...,...,...,...,...
178,MLB1992889189,YXIW04349,3 PARES 7A,active,13,29.0,31,0.935484,16.0,-12.0
179,MLB3696018282,ZRTG53907,"1,5",active,7,1.0,31,0.032258,-6.0,-6.0
180,MLB2025558078,ZUIU17173,0.040,active,1,1.0,31,0.032258,-0.0,-0.0
181,MLB2770292345,ZUKG50028,2,active,37,28.0,31,0.903226,-9.0,-36.0


In [72]:
df_sales["days_available"].value_counts()

days_available
31    92
0     24
32    23
30     9
24     5
28     4
2      4
23     4
7      3
26     3
29     3
8      3
21     2
1      1
6      1
5      1
4      1
Name: count, dtype: int64

In [73]:
df_sales.sample(2)

,ml_code,ml_inventory_id,value_name,status,available_quantity_today,sold_quantity_sum,days_available,media_prod_days_available,period_send_fulfillment,today_send_fulfillment
67,MLB1435852259,TLCG38735,NaN,active,3,1.0,32,0.03125,-2.0,-2.0
162,MLB1813063328,UTKO14988,Preto,active,103,98.0,32,3.06250,-11.0,-99.0


In [74]:
x = "MLB1629379059"
y = df_sales[df_sales["ml_code"] == x]
y

,ml_code,ml_inventory_id,value_name,status,available_quantity_today,sold_quantity_sum,days_available,media_prod_days_available,period_send_fulfillment,today_send_fulfillment
90,MLB1629379059,ZOQD52873,NaN,active,60,288.0,24,12.0,300.0,-48.0


In [75]:
condf(df_sales, "ml_code", "MLB1992541482")

,ml_code,ml_inventory_id,value_name,status,available_quantity_today,sold_quantity_sum,days_available,media_prod_days_available,period_send_fulfillment,today_send_fulfillment
100,MLB1992541482,CLMF99895,0.12,active,11,15.0,31,0.483871,4.0,-10.0
135,MLB1992541482,LUHT04035,0.09,active,19,4.0,31,0.129032,-15.0,-18.0
169,MLB1992541482,XGNL04251,0.11,active,5,1.0,31,0.032258,-4.0,-4.0


Pergunta:

Caso em seja necessário enviar produtos de um kit e apenas um dos produtos estiver em falta, o que fazer?